In [1]:
import numpy as np
import pandas as pd

## Data loading

In [2]:
data_path = './data'

In [3]:
products=pd.read_csv(data_path + '/products.csv')
stores=pd.read_csv(data_path +'/stores.csv')
transactions=pd.read_csv(data_path+ '/transactions.csv')

In [4]:
transactions.shape

(1096000, 4)

### Data preparation

In [5]:
agg_transactions = (
    transactions
        .groupby(['product_id', 'date'])
    ['nb_sold_pieces']
        .agg('sum')
        .reset_index()
)

In [6]:
agg_transactions['Week'] = pd.to_datetime(agg_transactions['date']).dt.week
agg_transactions['year'] = pd.to_datetime(agg_transactions['date']).dt.year
agg_transactions=agg_transactions.groupby(
    ['product_id', 'Week', 'year'])['nb_sold_pieces'].agg('sum').reset_index()
agg_transactions

/usr/local/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  """Entry point for launching an IPython kernel.


,product_id,Week,year,nb_sold_pieces
0,1,1,2017,151
1,1,1,2018,128
2,1,1,2019,108
3,1,2,2017,130
4,1,2,2018,123
...,...,...,...,...
15695,100,51,2017,38
15696,100,51,2018,31
15697,100,52,2016,32
15698,100,52,2017,28


In [7]:
df = agg_transactions.merge(products, on= 'product_id')
df=df.drop(['product_name', 'color'], axis=1)
df

,product_id,Week,year,nb_sold_pieces,gross_price,supplier
0,1,1,2017,151,13.68,NaN
1,1,1,2018,128,13.68,NaN
2,1,1,2019,108,13.68,NaN
3,1,2,2017,130,13.68,NaN
4,1,2,2018,123,13.68,NaN
...,...,...,...,...,...,...
15695,100,51,2017,38,80.82,John
15696,100,51,2018,31,80.82,John
15697,100,52,2016,32,80.82,John
15698,100,52,2017,28,80.82,John


## Feature engineering

In [8]:
df['date'] = pd.to_datetime(df['Week'].astype(str)+df['year'].astype(str).add('-1'),format = "%W%Y-%w")
df['quarter'] = df['date'].dt.quarter
df[f'lag_target_1W'] = df.groupby('product_id')['nb_sold_pieces'] \
    .transform(lambda x:x.shift(1))
df[f'lag_target_1W']=df[f'lag_target_1W'].fillna(method='bfill')
df[f'lag_target_2W'] = df.groupby('product_id')['nb_sold_pieces'] \
    .transform(lambda x:x.shift(2)).fillna(method='bfill')
lag=3
df[f'lag_target_{lag}W'] =df.groupby('product_id')['nb_sold_pieces'].shift(lag)
df[f'lag_target_{lag}W']=df[f'lag_target_{lag}W'].fillna(method='bfill')
lag=4
df[f'lag_target_{lag}W'] = df.groupby('product_id')['nb_sold_pieces'].shift(lag)
df[f'lag_target_{lag}W']=df[f'lag_target_{lag}W'].fillna(method='bfill')
lag=5
df[f'lag_target_{lag}W'] = (df.groupby('product_id')['nb_sold_pieces']
    .transform(lambda x: x.shift(lag)).fillna(method='bfill'))
df[f'lag_target_{lag}W']=df[f'lag_target_{lag}W'].fillna(method='bfill')
df[f'rolling_mean_3W'] = np.mean(df[
    ['lag_target_1W', 'lag_target_2W', 'lag_target_3W']], axis=1)
df[f'rolling_mean_4W'] = np.mean(df[['lag_target_1W', 'lag_target_2W', 'lag_target_3W', 'lag_target_4W']],
                                 axis=1)
df[f'rolling_mean_5W'] = np.mean(
    df[['lag_target_1W', 'lag_target_2W', 'lag_target_3W', 'lag_target_4W', 'lag_target_5W']], axis=1)

df['sin_week'] = np.sin((df['Week'] - 1) * np.pi *2 / (52+71 / 400))
df['cos_week'] = np.cos((df['Week']-1) * 2*np.pi /(52+(71 / 400)))
df['supplier'] = df['supplier'].fillna('UNKNOWN')
df['supplier'] = df['supplier'].astype('category').cat.codes
df

,product_id,Week,year,nb_sold_pieces,gross_price,supplier,date,quarter,lag_target_1W,lag_target_2W,lag_target_3W,lag_target_4W,lag_target_5W,rolling_mean_3W,rolling_mean_4W,rolling_mean_5W,sin_week,cos_week
0,1,1,2017,151,13.68,3,2017-01-02,1,151.0,151.0,151.0,151.0,151.0,151.000000,151.00,151.0,0.000000,1.000000
1,1,1,2018,128,13.68,3,2018-01-01,1,151.0,151.0,151.0,151.0,151.0,151.000000,151.00,151.0,0.000000,1.000000
2,1,1,2019,108,13.68,3,2019-01-07,1,128.0,151.0,151.0,151.0,151.0,143.333333,145.25,146.4,0.000000,1.000000
3,1,2,2017,130,13.68,3,2017-01-09,1,108.0,128.0,151.0,151.0,151.0,129.000000,134.50,137.8,0.120129,0.992758
4,1,2,2018,123,13.68,3,2018-01-08,1,130.0,108.0,128.0,151.0,151.0,122.000000,129.25,133.6,0.120129,0.992758
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15695,100,51,2017,38,80.82,1,2017-12-18,4,29.0,38.0,35.0,22.0,29.0,34.000000,31.00,30.6,-0.259219,0.965819
15696,100,51,2018,31,80.82,1,2018-12-17,4,38.0,29.0,38.0,35.0,22.0,35.000000,35.00,32.4,-0.259219,0.965819
15697,100,52,2016,32,80.82,1,2016-12-26,4,31.0,38.0,29.0,38.0,35.0,32.666667,34.00,34.2,-0.141319,0.989964
15698,100,52,2017,28,80.82,1,2017-12-25,4,32.0,31.0,38.0,29.0,38.0,33.666667,32.50,33.6,-0.141319,0.989964


## Train/test split

In [9]:
from sklearn.model_selection import train_test_split

df_train,df_preds = train_test_split(df,test_size=0.20)

In [10]:
y_train=df_train.pop('nb_sold_pieces')

X_train= df_train[[c for c in df_train.columns if c != 'period']].drop('date',axis=True, inplace=False)
y_test = df_preds.pop('nb_sold_pieces')
x_test =df_preds[[c for c in df_train.columns if c != 'period']].drop(['date'], axis=True)

# Model training

In [11]:
from sklearn.ensemble import RandomForestRegressor
model= RandomForestRegressor(n_estimators = 100)

model.fit(X_train,y_train)

RandomForestRegressor()

In [12]:
y_test_preditions = model.predict(x_test)

In [13]:
from sklearn.metrics import r2_score
print("R2 SCORE : %f" % r2_score(y_test, y_test_preditions))
wape= np.sum(np.abs(y_test- y_test_preditions)/np.sum(y_test))


print("wape score on test: %f" % wape)

R2 SCORE : 0.994080
wape score on test: 0.014839
